# Simulación del tráfico en un carril.

Primero se simulará para dos carros. La ecuación que simulará la velocidad de los vehículos es el mínimo entre

$$\dot{x_i} = \frac{1}{T_i}(x_{i-1}-x_i -\frac{l_i + l_{i-1}}{2}) 
$$ 

y

$$ \dot{x_i} = v_{max_{i}} $$

In [10]:
#Llamando al paquete de Taylor

using TaylorSeries

In [45]:
#Definiendo parámetros. Sólo para fijar ideas.

x1_0 = 0.0;
x2_0 = pi;
l1 = pi/16;
l2 = l1;
V1 = pi/4;
V2 = pi/2;

per1=1;
per2=1;



A continuación se escribe la función `modulo2pi` que encuentra la diferencia entre dos ángulos en módulo 2$\pi$

In [12]:
function modulo2pi(α,β)
    #Esta función calcula la diferencia entre dos ángulos y la devuelve 
    #módulo 2π.
    δ=α-β 
    
    if δ<0
        δ = 2π + δ
    else 
        δ = δ 
    end
    
    return δ
end

modulo2pi (generic function with 1 method)

In [13]:
#La siguiente función calcula los coeficientes de la velocidad de seguridad

function velocidad_sec{R<:Real }(T::R, car::Taylor1{R}, frontcar::Taylor1{R} , lfcar::R)
    k = max(car.order, frontcar.order)
    
    vel = taylor1_variable(Float64, k)
     
    vel.coeffs[1] = 1/T * modulo2pi(frontcar.coeffs[1]-lfcar,car.coeffs[1])
    
    for i = 2 : k
        vel.coeffs[i] = 1/T * (frontcar.coeffs[i]-car.coeffs[i])
    end
        
    
    return vel
end



velocidad_sec (generic function with 1 method)

Ahora se definen un par de variables tipo `Taylor1` que tendrá información de las series de taylor de las funciones $x_1 (t)$ y $x_2 (t)$.

Ahora, a encontrar los siguientes coeficientes. Usando que $$ x_{n+1} = f_n /(n+1) $$

Es momento de avanzar. definamos la épsilon y la función para calcular el siguiente paso.

Sabemos que el paso siguiente está dado por $ h = \min[\, (\epsilon/x^{[p-1]})^{1/(p-1)}, (\epsilon/x^{[p]})^{1/p}\, ], $

In [31]:
function stepsize{T<:Real}(x::Taylor1{T}, epsilon::Float64)
    ord = x.order
    h = Inf
    
    if ord > 1
        
        for k in [ord-1, ord]
            kinv = 1.0/k
            aux = abs( x.coeffs[k+1] )
            h = min(h, (epsilon/aux)^kinv)
        end
    
    elseif ord == 1
        
        
        
        aux = abs( x.coeffs[1.0])
        
        h = min(h, (epsilon/aux))
        
    end
        
    
    
    return h
end

stepsize (generic function with 1 method)

Necesitamos una función que sólo con las condiciones iniciales (y otros parámetros) , calcule las series de dos coches.

In [15]:
function series(x10,x20,T1,T2,L1,L2,V1,V2)
   
    
    vel1 = 1/T1 * modulo2pi(x20-L2,x10)
    
    vel2 = 1/T2 * modulo2pi(x10-L1,x20)
    #Velocidades de seguridad.
    
    if vel1 < V1 && vel2 < V2
    
        x1 = taylor1_variable(Float64,20)
        x2 = taylor1_variable(Float64,20)
    
        x1.coeffs[1] = x10
        x1.coeffs[2] = 0
    
        x2.coeffs[1] = x20
        x2.coeffs[2] = 0
        
        for i = 2:20
            x1.coeffs[i] = velocidad_sec(T1,x1,x2,L2).coeffs[i-1]/i
            x2.coeffs[i] = velocidad_sec(T2,x2,x1,L1).coeffs[i-1]/i
        end
    
    
    elseif vel1 < V1 && vel2 > V2
        
        x1 = taylor1_variable(Float64,20)
        x2 = taylor1_variable(Float64,1)
        
        x1.coeffs[1] = x10
        x1.coeffs[2] = 0
        
        x2.coeffs[1] = x20
        
        
        x2.coeffs[2] = V2
        for i = 2:20
            x1.coeffs[i] = velocidad_sec(T1,x1,x2,L2).coeffs[i-1]/i
        end
        
    
    

    
    elseif vel1 > V1 && vel2 < V2
        
        x1 = taylor1_variable(Float64,1)
        x2 = taylor1_variable(Float64,20)
        
        x1.coeffs[1] = x10
        x1.coeffs[2] = V1
        
        x2.coeffs[1] = x20
        x2.coeffs[0] = 0
        
        for i = 2:20
            x2.coeffs[i] = velocidad_sec(T2,x2,x1,L1).coeffs[i-1]/i
        end
        
        
    
    
    elseif vel1 > V1 && vel2 > V2
        x1 = taylor1_variable(Float64,1)
        x2 = taylor1_variable(Float64,1)
        
        x1.coeffs[1] = x10
        x1.coeffs[2] = V1
        
        x2.coeffs[1] = x20
        x2.coeffs[2] = V2
        
    end
    
    return x1, x2
    
end
    
        
    
    
    
    

series (generic function with 1 method)

In [46]:

#t1 y t2 son vectores donde se guardarán los tiempos donde se evalúa la función.
t1 = Float64[];
push!(t1,0);

#x1 y x2 son vectores donde se guardarán los valores de las funciones x1(t) y x2(t)

x1 = Float64[];
push!(x1,x1_0);
x2 = Float64[];
push!(x2,x2_0);




for n in 2:100000
    
    x1Taylor = series(x1_0,x2_0,per1,per2,l1,l2,V1,V2)[1]
    x2Taylor = series(x1_0,x2_0,per1,per2,l1,l2,V1,V2)[2]
    
    h1 = stepsize(x1Taylor,1.0e-10)
    
    h2 = stepsize(x2Taylor,1.0e-10)
    h = min(h1,h2)
    
    println(h)
    
    push!(t1,t1[n-1] + h)
    
    x1_0 = evaluate(x1Taylor,h)
    push!(x1,x1_0)
    x2_0 = evaluate(x2Taylor,h)
    push!(x2,x2_0)
    
end
    
    
    
    
    

In [47]:
x1

100000-element Array{Float64,1}:
 0.0       
 2.5e-11   
 5.0e-11   
 7.5e-11   
 1.0e-10   
 1.25e-10  
 1.5e-10   
 1.75e-10  
 2.0e-10   
 2.25e-10  
 2.5e-10   
 2.75e-10  
 3.0e-10   
 ⋮         
 2.4997e-6 
 2.49972e-6
 2.49975e-6
 2.49977e-6
 2.4998e-6 
 2.49982e-6
 2.49985e-6
 2.49987e-6
 2.4999e-6 
 2.49992e-6
 2.49995e-6
 2.49997e-6

In [33]:
h = stepsize(x2,1.0e-20)

6.366197723675813e-21

In [40]:
t1

100000-element Array{Float64,1}:
 0.0     
 0.745132
 0.745132
 0.745132
 0.745132
 0.745132
 0.745132
 0.745132
 0.745132
 0.745132
 0.745132
 0.745132
 0.745132
 ⋮       
 0.745166
 0.745166
 0.745166
 0.745166
 0.745166
 0.745166
 0.745166
 0.745166
 0.745166
 0.745166
 0.745166
 0.745166

In [48]:
Pkg.add("Gadfly")

INFO: Cloning cache of ArrayViews from git://github.com/JuliaLang/ArrayViews.jl.git
INFO: Cloning cache of Calculus from git://github.com/johnmyleswhite/Calculus.jl.git
INFO: Cloning cache of Codecs from git://github.com/dcjones/Codecs.jl.git
INFO: Cloning cache of Compose from git://github.com/dcjones/Compose.jl.git
INFO: Cloning cache of Contour from git://github.com/tlycken/Contour.jl.git
INFO: Cloning cache of DataArrays from git://github.com/JuliaStats/DataArrays.jl.git
INFO: Cloning cache of DataFrames from git://github.com/JuliaStats/DataFrames.jl.git
INFO: Cloning cache of DataStructures from git://github.com/JuliaLang/DataStructures.jl.git
INFO: Cloning cache of Distances from git://github.com/JuliaStats/Distances.jl.git
INFO: Cloning cache of Distributions from git://github.com/JuliaStats/Distributions.jl.git
INFO: Cloning cache of DualNumbers from git://github.com/JuliaDiff/DualNumbers.jl.git
INFO: Cloning cache of GZip from git://github.com/JuliaLang/GZip.jl.git
INFO: Cloni

In [ ]:
using Gadfly

INFO: Precompiling module Gadfly...
Expr(:import, :Patchwork)::Any
  ** incremental compilation may be broken for this module **



In [ ]:
plot(x=t1,y=x1)